In [ ]:
import hdf5_getters
import h5py
import glob
import pandas as pd

In [ ]:
# Import Y-order song list
f = h5py.File("data/Y.hdf5", "r")
songs = [n[0].decode('utf-8') for n in f['songs']]
f.close()

In [ ]:
# Get filenames
X_files = glob.glob("/mnt/data/X/*/*/*.h5")
Y_files = glob.glob("/mnt/data/Y/*/*/*.h5")
Z_files = glob.glob("/mnt/data/Z/*/*/*.h5")
files = X_files+Y_files+Z_files

In [ ]:
# Get data
mfcc = []
artist = []
title = []
song_id = []

count = 0
for dset in files:
    with hdf5_getters.open_h5_file_read(dset) as file:
        track_id = hdf5_getters.get_song_id(file).decode("utf-8")
        if track_id in songs:
            features = hdf5_getters.get_segments_timbre(file)
            if features.shape[0] >= 200:
                mfcc.append(features[:200])
                artist.append(hdf5_getters.get_artist_name(file).decode("utf-8"))
                title.append(hdf5_getters.get_title(file).decode("utf-8"))
                song_id.append(track_id)
                count += 1
                if count % 1000 == 0:
                    print(count)

In [ ]:
# Store X as hdf5
f = h5py.File("data/X_XYZ.hdf5", "w")
f.create_dataset("mfcc", data=mfcc)
ascii_artist = [n.encode("ascii", "ignore") for n in artist]
f.create_dataset('artist', data=ascii_artist)
ascii_title = [n.encode("ascii", "ignore") for n in title]
f.create_dataset('title', data=ascii_title)
ascii_song_id = [n.encode("ascii", "ignore") for n in song_id]
f.create_dataset('song_id', data=ascii_song_id)
f.close()